In [ ]:
def statistics(ws):
    import numpy as np

    weights = np.array(ws)
    # log returns:
    rets = np.log(data / data.shift(1))
    # expected portfolio return:
    pret = np.sum(rets.mean() * weights) * 252 
    # expected portfolio volatility:
    pvol = np.sqrt(np.dot(weights.T, np.dot(rets.cov() * 252, weights))) 
    # for simplicity we assume risk free short rate = 0:
    rf = 0
    # expected excess of the portfolio / expected standard deviation of the portfolio
    sharpe_ratio = (pret - rf) / pvol
    return np.array([pret, pvol, pret / pvol, sharpe_ratio])

In [ ]:
def min_func_port(weights):
    return statistics(weights)[1]

In [ ]:
def efficient_frontier_portfolios():
    import scipy.interpolate as sci
    import scipy.optimize as sco
    import numpy as np
    
    # weights boundaries between 0 and 1
    bnds = tuple((0, 1) for x in range(noa))
    # constraint that all weights add up to 1
    cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1 })
    
    trets = np.linspace(0.0, 0.25, 50)
    tvols = []
    # for each return level find those portfolio weights that lead to the minimum volatility:
    for tret in trets:
        cons = ({'type': 'eq', 'fun': lambda x: statistics(x)[0] - tret}, 
                {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
        initial_weights = noa * [ 1. / noa ,]
        res = sco.minimize(min_func_port, initial_weights, method='SLSQP', bounds=bnds , constraints=cons)
        tvols = np.append(tvols, res['fun']) 
        tvols = np.array(tvols)
    # select efficient frontier portfolios- with a higher return than the absolute minimum variance portfolio:
    ef_ind = np.argmin(tvols)
    ef_vols = tvols[ef_ind:]
    ef_rets = trets[ef_ind:]
    
    # interpolate efficient frontier curve:
    tck = sci.splrep(ef_vols, ef_rets)
    return tck

In [ ]:
def capital_market_line(tck, risk_free_rate):
    # Determine an efficient portfolio of risky assets and then add the riskless asset to the mix.
    # We are looking for is a function f(x)=a+b*x describing the line that passes through 
    # the riskless asset (a=risk_free_rate) and that is tangent to the efficient frontier.
    import scipy.interpolate as sci
    import scipy.optimize as sco
    
    def f(x):
        # efficient frontier function (spline approximation)
        return sci.splev(x, tck, der=0)
    
    def df(x):
        # first derivative of efficient frontier function required to find tangent to the efficient frontier
        return sci.splev(x, tck, der=1)
    
    def equations(p, rf=risk_free_rate):
        eq1 = rf - p[0]
        eq2 = rf + p[1] * p[2] - f(p[2]) 
        eq3 = p[1] - df(p[2]) 
        return eq1 , eq2 , eq3
    
    # find cml by solving equations:
    opt = sco.fsolve(equations, [risk_free_rate, 20, 0.2])
    return opt

In [ ]:
def load_data(symbols):
    import pandas as pd
    from dao import InvestDao
    
    dao = InvestDao()
    data = pd.DataFrame()
    for sym in symbols:
        rows = dao.sql("SELECT close FROM stock_quotes WHERE dtyymmdd > '2016-06-01' AND ticker = '%s' ORDER BY dtyymmdd" % sym)
        data[sym] = [float(r[0]) for r in rows]
    data.columns = symbols
    return data

In [ ]:
symbols = ['KGHM', 'BZWBK', 'PGNIG', 'PKOBP', 'CCC', 'CYFRPLSAT', 'ENERGA']
data = load_data(symbols)
noa = len(symbols)

In [ ]:
efp = efficient_frontier_portfolios()

In [ ]:
risk_free_rate = 0.01
cml = capital_market_line(efp, risk_free_rate)

In [ ]:
print("Capital market line is: y = %s + %s * x" % (cml[0], cml[1]))